<a href="https://colab.research.google.com/github/Gyeong-Hyeon/AI_Project/blob/main/Section2/AI_01_%EC%97%BC%EA%B2%BD%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade category_encoders
!pip install eli5
!pip install pdpbox
!pip install shap

In [ ]:
from IPython.display import display
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import TargetEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, roc_curve, roc_auc_score
import eli5
from eli5.sklearn import PermutationImportance
from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
import shap
import warnings

#**1. 데이터 전처리 및 랭글링** 

##**1) 데이터 준비하기**

In [24]:
#데이터를 불러옵니다.
patient_21 = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQDLQ5Bdj7692BU0e1EwsEyG2h5cpyt8pcrXOQ3uOQixIQ4Qbrn5d6jX5WnxeNpWr3mrECiXgPWfkWf/pub?output=xlsx')
vacc_21 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTr60lC-MNfPA_PgY7NjtFPuV_uEgN5uETLUSJkeoruTZyluDhxzY8D0JVF4bijkbFJv6E2QNC2zZku/pub?output=csv')
symp_21 = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTfBuM4QzwTsL6hlfo33mY97P0PPNbeAtvohvnGo0MQKkIfJPzBGRRYYA_0bhmL6dJPMVk82zB2zDRX/pub?output=xlsx')
patient_20 = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vSaR-9rRkFWSJYTRsI6KLXOBANnOpt5oNyJLd8rhnBE7dWslQ2QP69E5bfUO-cfXkdn2Elpvi549RcF/pub?output=xlsx')
vacc_20 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSbKt4I1BVWLqB6FyWAfcWr7lCFctvWd4b4C7YHNtcA9znBT-cs15Q1NM7ToBs5aczuX2_hK28KO6wA/pub?output=csv')
symp_20 = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vR39RIbZ2NKL5iVHh2v24CXc6PZoUjD5GkPxj5FRPa-9lCagP_VCa1b3EB77mfpTYnbYQRAoyzozRsr/pub?output=xlsx')

dfs = [patient_21, patient_20, vacc_21, vacc_20, symp_21, symp_20]

for df in dfs:
      print(df.shape)
      display(df.head())
      print('\n')

(5351, 33)


,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916710,2021-01-01 00:00:00,MO,23.0,23.0,NaN,F,NaN,NaN,NaN,Y,NaN,Y,NaN,NaN,NaN,U,12/29/2020,2021-01-01 00:00:00,3.0,PVT,NaN,Synthroid,NaN,Hypothyroidism,NaN,NaN,2,2021-01-01 00:00:00,NaN,NaN,Y,NKDA
1,916720,2021-01-01 00:00:00,NY,23.0,23.0,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/31/2020,12/31/2020,0.0,PVT,NaN,NaN,NaN,NaN,unknown,NaN,2,2021-01-01 00:00:00,NaN,NaN,Y,NaN
2,916741,2021-01-01 00:00:00,AR,68.0,68.0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,12/21/2020,12/22/2020,1.0,PVT,NaN,phenobarbital 60mg HS hydroxychloroquin 400mg...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,2021-01-01 00:00:00,NaN,NaN,NaN,bee stings
3,916772,2021-01-01 00:00:00,GA,55.0,55.0,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,12/22/2020,12/26/2020,4.0,PUB,NaN,"Propranolol,zoloft,asa,Lisinipril, Crestor, Pr...",NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,2021-01-01 00:00:00,NaN,Y,NaN,Codeine
4,916790,2021-01-01 00:00:00,TN,52.0,52.0,NaN,F,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,U,12/26/2020,12/26/2020,0.0,PVT,NaN,"Estradiol, topiramate, Emgality, sumatriptan, ...",NaN,"Asthma, migraines",NaN,NaN,2,2021-01-01 00:00:00,NaN,NaN,Y,"Sulfa, shellfish"




(9534, 42)


,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,902440,12/15/2020,AZ,35.0,35.0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,12/15/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,902446,12/15/2020,WV,55.0,55.0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/15/2020,12/15/2020,0,OTH,NaN,"Novasc, Hydrochlorothiazide, synthroid, lisino...",none,"Hypertension, sleep apnea, hypothyroidism",NaN,NaN,2,12/15/2020,NaN,NaN,NaN,"Contrast Dye IV contrast, shellfish, strawberry",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,902448,12/15/2020,IL,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,N,2015-10-08 00:00:00,NaN,NaN,UNK,NaN,NaN,Routine health maintenance,NaN,NaN,US0095075132012USA004082,2,12/15/2020,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,902449,12/15/2020,NC,51.0,51.0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,11/30/2020,11/30/2020,0,UNK,NaN,NaN,Crohn's disease; Ehlers-Danlos syndrome; Fibro...,NaN,NaN,USGLAXOSMITHKLINEUS2020AM,2,12/15/2020,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,902450,12/15/2020,GA,67.0,67.0,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,11/23/2020,11/23/2020,0,UNK,NaN,NaN,NaN,NaN,NaN,USGLAXOSMITHKLINEUS202023,2,12/15/2020,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




(5471, 8)


...

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,855017,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,SYR,LA,ZOSTER (SHINGRIX)
1,855018,UNK,UNKNOWN MANUFACTURER,NaN,1,IM,RA,VACCINE NOT SPECIFIED (OTHER)
2,855019,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,NaN,NaN,ZOSTER (SHINGRIX)
3,855020,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,UNK,IM,LA,ZOSTER (SHINGRIX)
4,855021,VARZOS,GLAXOSMITHKLINE BIOLOGICALS,NaN,2,IM,RA,ZOSTER (SHINGRIX)




(5351, 1)


,SYMPTOM_TEXT
0,"Acute appendicitis, onset morning of 1/1/2021 ..."
1,Patient made statements that he was having a h...
2,"on dec 22 I felt some myalgias, chills, fatigu..."
3,Vaccine on 12/22/2020 and started feeling bad ...
4,"Flushing, sweating, increased heart rate proce..."




(9534, 1)


,SYMPTOM_TEXT
0,C/O Headache
1,"felt warm, hot and face and ears were red and ..."
2,stroke; neuropathy; post herpetic neuralgia; c...
3,the worst RA pain ever; flu-like symptoms; fev...
4,Fluid in Lungs; foot Swelling; Difficulty brea...


In [27]:
patient_20.DIED.unique()

array([nan, 'Y', ' synthroid"',
       ' while walking back to work on the way to work unit noticed tingling on the L side of her lower lip and a few inches back along jaw.  It persisted so she returned to the vaccine clinic for evaluation.  She was evaluated by a physician and had a normal cranial nerve and motor exam other than diminished sensation CN5- no change in appearance of lip.  There was no other reaction at vaccination site or change in symptoms during a further 30 minute observation period.  She was released back to work.  Checkin after a few hours the tingling had improved."',
       'Depression', 'shortness of breath', 'heart palpitations',
       ' via an unspecified route of administration on 16Dec2020 14:15 at single dose for immunisation. The patient medical history and patient\'s concomitant medications were not reported.  The patient experienced injection site pain on 17Dec2020 07:00. The vaccine was administered at Hospital Facility. The patient did not receive any

In [ ]:
#2020년 환자 데이터에 'Unnamed'라는 공란 컬럼을 삭제하겠습니다.
patient_20 = patient_20.drop(columns=[col for col in patient_20.columns if 'Unnamed' in col])

#2020년과 2021년 데이터를 합쳐줍니다.
patient = pd.concat([patient_20,patient_21])
vacc = pd.concat([vacc_20,vacc_21])


In [20]:
patient.DIED.unique()

array([nan, 'Y', ' synthroid"',
       ' while walking back to work on the way to work unit noticed tingling on the L side of her lower lip and a few inches back along jaw.  It persisted so she returned to the vaccine clinic for evaluation.  She was evaluated by a physician and had a normal cranial nerve and motor exam other than diminished sensation CN5- no change in appearance of lip.  There was no other reaction at vaccination site or change in symptoms during a further 30 minute observation period.  She was released back to work.  Checkin after a few hours the tingling had improved."',
       'Depression', 'shortness of breath', 'heart palpitations',
       ' via an unspecified route of administration on 16Dec2020 14:15 at single dose for immunisation. The patient medical history and patient\'s concomitant medications were not reported.  The patient experienced injection site pain on 17Dec2020 07:00. The vaccine was administered at Hospital Facility. The patient did not receive any

* Patient: 환자의 정보가 담긴 데이터 셋
* Vacc: 백신 정보가 담긴 데이터 셋

💊VAERS_ID (고유 번호)를 기준으로 두 데이터를 합칠 수 있습니다.
각 데이터별로 중복된 ID가 있는지 먼저 확인 해보겠습니다.

In [3]:
#백신 데이터를 Covid 백신으로만 소팅합니다

mask = vacc['VAX_TYPE'].isin(['COVID19'])
covid_vac = vacc[mask]
print('Length of Covid vaccine dataset:',len(covid_vac))

#Covid 백신 데이터 셋의 VAERS_ID열에 중복된 데이터가 있는지 확인합니다.
print('Duplicated VAERS ID in Covid vaccine dataset:',covid_vac.duplicated(['VAERS_ID']).sum())
mask = covid_vac.duplicated(['VAERS_ID'])
covid_duplicated = covid_vac[mask]
covid_duplicated['VAERS_ID'].unique()

Length of Covid vaccine dataset: 14337
Duplicated VAERS ID in Covid vaccine dataset: 78


array([ 905340,  906428,  907330,  907837,  909370,  909520,  911085,
        912442,  912896,  913038,  913869,  914017,  914458,  937480,
        938126,  938576,  943614,  944595,  945504,  946663,  948418,
        949732,  950911,  957227,  959928,  962110,  963587,  964617,
        967274,  968195,  970198,  970515,  971567,  971939,  973816,
        974177,  975206,  978768,  983425,  984929,  988246,  989556,
        990109,  990118,  990694,  990702,  990718,  991686,  992082,
        992774,  994007,  995346,  995419,  996577, 1000418, 1000733,
       1000849, 1003553, 1005737, 1006745, 1007357, 1007628, 1007928,
       1008767, 1009424, 1011689, 1011707, 1011983, 1015253, 1015465,
       1015921, 1016770, 1016907, 1019670, 1020144, 1020227, 1022397,
       1024343])

* **백신 데이터 중복 VAERS_ID:** 46개
* **중복되는 경우**

1. 같은 환자에게 다른 종류의 백신 접종 ⭐

2. 같은 환자에게 다른 Lot의 백신 접종 → drop

3. 백신 루트나 백신 접종 지역이 다름 → drop

In [ ]:
ids = [905340,  906428,  907330,  907837,  909370,  909520,  911085, 912442,  912896,  913038,  913869,  914017,  914458,  937480, 938126,  938576,  943614,  944595,  945504,  946663,  948418,
        949732,  950911,  957227,  959928,  962110,  963587,  964617, 967274,  968195,  970198,  970515,  971567,  971939,  973816, 974177,  975206,  978768,  983425,  984929,  988246,  989556,
        990109,  990118,  990694,  990702,  990718,  991686,  992082, 992774,  994007,  995346,  995419,  996577, 1000418, 1000733, 1000849, 1003553, 1005737, 1006745, 1007357, 1007628, 1007928,
       1008767, 1009424, 1011689, 1011707, 1011983, 1015253, 1015465, 1015921, 1016770, 1016907, 1019670, 1020144, 1020227, 1022397, 1024343]

for id in ids:
  mask = covid_vac['VAERS_ID'].isin([id])
  mask = covid_vac[mask]
  if len(mask['VAX_NAME'].unique()) == 1:
    pass
  else:
    print(mask[['VAERS_ID', 'VAX_DOSE_SERIES','VAX_NAME']],'\n')

       VAERS_ID VAX_DOSE_SERIES                             VAX_NAME
51963    912896               2          COVID19 (COVID19 (MODERNA))
51964    912896               1  COVID19 (COVID19 (PFIZER-BIONTECH)) 

      VAERS_ID VAX_DOSE_SERIES                             VAX_NAME
2524    967274             UNK          COVID19 (COVID19 (MODERNA))
2525    967274             UNK  COVID19 (COVID19 (PFIZER-BIONTECH)) 



* 같은 백신을 투약한 VAERS_ID: 44개 (drop)

* 다른 백신을 투약한 VAERS_ID: 912896, 967274

In [ ]:
duplicated = patient[patient['VAERS_ID'].isin([912896,967274])]
for id,symp in zip(duplicated['VAERS_ID'],duplicated['SYMPTOM_TEXT']):
  print('Patient#',id, symp)

Patient# 912896 patient received a dose of Moderna vaccine after receiving the Pfizer vaccine.
Patient# 967274 I was pregnant and my baby died two days after I took it and I got really sick


912896은 접종 후에 대한 증상이나 사망 여부 등 특별한 정보가 없으므로 삭제하겠습니다.

967274는 태아가 죽은 것으로 확인 되나, 모더나와 화이자 중 어느 백신의 영향인지 알 수 없으므로 삭제하겠습니다.

In [ ]:
print('Duplicated VAERS ID in Patient dataset:',patient.duplicated(['VAERS_ID']).sum())

Duplicated VAERS ID in Patient dataset: 0


In [4]:
covid = covid_vac[['VAERS_ID','VAX_MANU']]
covid = covid.drop_duplicates(['VAERS_ID'])
covid = pd.merge(covid, patient, on = ['VAERS_ID'], how = 'inner')
print('기존 환자 수:',len(patient),'\nCovid백신을 접종한 환자 수:',len(covid))

기존 환자 수: 14885 
Covid백신을 접종한 환자 수: 14234


##**2) 데이터 전처리**


1. 결측치 정리 & 카디널리티 조정

2. 타겟 및 새로운 특성 만들기

3. 필요 없는 column drop



###1. 결측치 정리

In [ ]:
covid.isnull().sum()

VAERS_ID            0
VAX_MANU            0
RECVDATE            0
STATE            1772
AGE_YRS          1372
CAGE_YR          3536
CAGE_MO         14192
SEX                 0
RPT_DATE        14054
SYMPTOM_TEXT        0
DIED            13410
DATEDIED        13474
L_THREAT        13609
ER_VISIT        14202
HOSPITAL        12375
HOSPDAYS        13028
X_STAY          14211
DISABLE         13914
RECOVD           1139
VAX_DATE          587
ONSET_DATE        700
NUMDAYS          1148
LAB_DATA         7097
V_ADMINBY          12
V_FUNDBY        14035
OTHER_MEDS       5645
CUR_ILL          7312
HISTORY          4749
PRIOR_VAX       13677
SPLTTYPE        10304
FORM_VERS          16
TODAYS_DATE       281
BIRTH_DEFECT    14195
OFC_VISIT       12309
ER_ED_VISIT     11058
ALLERGIES        6113
dtype: int64

In [5]:
previous = covid['AGE_YRS'].isnull().sum()
covid['AGE_YRS'] = np.where(pd.notnull(covid['AGE_YRS']) == True, covid['AGE_YRS'], covid['CAGE_YR'])
current = covid['AGE_YRS'].isnull().sum()
print('CAGE_YR을 통해', previous-current,'개의 결측값을 채웠으며, 현재 나이 특성의 결측치 갯수는',current,'개입니다.')

CAGE_YR을 통해 78 개의 결측값을 채웠으며, 현재 나이 특성의 결측치 갯수는 1294 개입니다.


성별의 경우, 결측치는 없으나 'U'가 undefinded이므로 결측치라고 생각할 수 있습니다. 따라서 symptom을 통해 환자의 성별을 추측해보겠습니다.

In [6]:
previous = covid.SEX.isin(['U']).sum()
covid['split_symp'] = covid.SYMPTOM_TEXT.str.split(' ')
new_sex = []
for symptom, sex in zip(covid['split_symp'],covid['SEX']):
  if 'foetal' in symptom or 'Foetal' in symptom or 'fetal' in symptom or 'fetus' in symptom or 'Female' in symptom:
    new_sex.append(sex)
  elif sex == None or sex == 'U':
    if 'she' in symptom or 'She' in symptom or 'woman' in symptom or 'female' in symptom or 'Female' in symptom:
      new_sex.append('F')
    elif 'he' in symptom or 'He' in symptom or 'man' in symptom or 'male' in symptom or 'Male' in symptom:
      new_sex.append('M')
    else:
      new_sex.append(sex)
  else: 
    new_sex.append(sex)

covid['SEX'] = new_sex
current = covid.SEX.isin(['U']).sum()
print('증상 컬럼을 통해',previous-current,'개의 성별을 확인했습니다.')   

증상 컬럼을 통해 60 개의 성별을 확인했습니다.


백신 투약 날짜의 결측치는 on set date(보고된 날짜)로 채우고 카디널리티를 낮추기 위해 월로 변경하겠습니다.

In [16]:
covid['ONSET_DATE'].unique()

array(['12/15/2020', '12/14/2020', '12/16/2020', '12/01/2020',
       'Amoxicillin, Bactrim, Erythromycin, Augmentin', '12/17/2020',
       '12/18/2020', '11/17/2020', 'Osteoporosis', nan, '11/16/2020', 'Y',
       '12/19/2020', '12/20/2020', '12/21/2020', '12/22/2020', 'N',
       '12/23/2020', '01/22/2020', '12/04/2020', '12/11/2020',
       '12/12/2020', 'nausea', '12/05/2020', '12/24/2020', '08/15/2020',
       '12/25/2020', '10/23/2020', '12/26/2020', '12/27/2020',
       '12/28/2020', '01/03/2021', '12/29/2020', '12/09/2020',
       '12/03/2020', '11/29/2020', '12/30/2020', 'Nickel', '12/31/2020',
       '08/28/2020', '10/28/2020', '01/01/2021', '01/02/2021',
       '01/04/2021', '09/02/2020', '01/05/2021', '01/06/2021',
       '01/07/2021', '01/08/2021', '01/09/2021', '01/10/2021',
       '01/11/2021', '01/03/2020', '01/12/2021', '01/13/2021',
       '01/14/2021', '01/15/2021', '01/16/2021', '01/17/1921',
       '01/17/2021', '01/18/2021', '01/19/2021', '01/12/2020',
       '01/

In [17]:
mask = covid[(covid['DIED'] != 'Y'))
mask = covid[mask]
mask

,VAERS_ID,VAX_MANU,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES,split_symp
61,902791,PFIZER\BIONTECH,12/16/2020,KS,51.0,51.0,NaN,F,NaN,:1039: Responded to notification of potential ...,"synthroid""",unknown,unknown,NaN,NaN,2,12/16/2020,NaN,NaN,Y,"Amoxicillin, Bactrim, Erythromycin, Augmentin",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[:1039:, Responded, to, notification, of, pote..."
375,903433,PFIZER\BIONTECH,12/18/2020,KS,57.0,57.0,NaN,F,NaN,Patient was seen at COVID Vaccine Clinic today...,Depression,Diabetes mellitus type I,Double vision,GERD (gastroesophageal reflux disease),Hearing loss,History of liver transplant,History of stroke (2008),Hyperlipidemia,Obstructive sleep apnea,OSA (obstructive sleep apnea),Osteoporosis,Pain,chronic,Psoriasis (2015),Psoriatic arthropathy,and Squamous cell skin cancer (07/10/2019). Sh...,"2015).""",NaN,NaN,2,12/18/2020,NaN,NaN,NaN,Sulfa antibiotics,NaN,"[Patient, was, seen, at, COVID, Vaccine, Clini..."
671,903838,PFIZER\BIONTECH,12/18/2020,KS,25.0,25.0,NaN,F,NaN,Notes 12/18/2020 ? ??? Subjective No chief c...,heart palpitations,"trouble swallowing""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/17/2020,12/17/2020,0,none,PVT,NaN,unknown,none,none,NaN,NaN,2,12/18/2020,NaN,NaN,"[Notes, , 12/18/2020, ?, , ???, Subjective, No..."
2513,906871,PFIZER\BIONTECH,12/22/2020,NaN,60.0,60.0,NaN,F,NaN,Progress Notes; Nurse Practitioner Cosign Need...,pt c/o flushing,dizziness and fatigue. Pt monitored for 45min...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/22/2020,12/22/2020,0,None,WRK,NaN,Omeprazole atenolo albuterol,NaN,NaN,NaN,NaN,2,12/22/2020,NaN,NaN,"[Progress, Notes;, Nurse, Practitioner, Cosign..."
3353,907944,PFIZER\BIONTECH,12/23/2020,KS,34.0,34.0,NaN,F,NaN,Progress Notes PA-C (Physician Assistant) ? ? ...,via an unspecified route of administration fro...,both from an unknown date and unknown if ongoi...,food,or other products. Concomitant medications wer...,the patient was not diagnosed with COVID-19. S...,the patient had not been tested for COVID-19. ...,the patient experienced injection site soreness,myalgias,chills,fatigue,nausea,loss of appetite,and enanthem (2 oral ulcers). No therapeutic ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,12/18/2020,12/19/2020,1,NaN,"[Progress, Notes, PA-C, (Physician, Assistant)..."
3371,907981,PFIZER\BIONTECH,12/23/2020,KS,46.0,46.0,NaN,F,NaN,Progress Notes 12/22/2020 ?? Subjective P...,then 24 hours after vaccine severe headache wh...,"aches""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,12/19/2020,12/19/2020,0,none,PVT,NaN,claritin 10 mg daily tylenol 1000 mg once afte...,none,none,NaN,NaN,2,12/23/2020,NaN,Y,"[Progress, Notes, , , , , 12/22/2020, , ??, Su..."
6725,912138,MODERNA,12/29/2020,CA,32.0,32.0,NaN,F,NaN,She arrived at 12:30 and was vaccinated shortl...,acid reflux,migraine with aura,plantar fasciitis,"tinnitus""",NaN,NaN,2,12/29/2020,NaN,NaN,Y,Compazine: dystonic reaction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[She, arrived, at, 12:30, and, was, vaccinated..."
7786,913340,MODERNA,12/30/2020,MO,27.0,27.0,NaN,F,NaN,"While sleeping, I woke up around 2:00 AM with ...","levothryroxine""",NaN,NaN,NaN,NaN,2,12/30/2020,NaN,NaN,NaN,Nickel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[While, sleeping,, I, woke, up, around, 2:00, ..."


In [12]:
previous = covid['VAX_DATE'].isnull().sum()

#날짜가 아닌 value들을 결측치로 변경해줍니다.
covid['VAX_DATE'] = covid['VAX_DATE'].str.replace([])
covid['ONSET_DATE'] = covid['ONSET_DATE'].str.replace()

#VAX_DAT와 ONSET_DATE 열을 날짜 형식으로 변경해줍니다.
covid['VAX_DATE'] = pd.to_datetime(covid['VAX_DATE'], format='%m/%d/%y %H:%M:%S', errors='raise')
covid['ONSET_DATE'] = pd.to_datetime(covid['ONSET_DATE'], format='%m/%d/%y %H:%M:%S', errors='raise')

#ONSET_DATE - NUMDAYS를 통해 결측치를 채웁니다. 만약 NUMDAYS가 빈칸인 경우, ONSET DATE로 VAC DATE를 채웁니다.
covid['VAX_DATE'] = np.where((pd.notnull(covid['VAX_DATE']) == True) &
                             (pd.notnull(covid['NUMDAYS']) == False) &
                             (pd.notnull(covid['ONSET_DATE']) == False)
                             , covid['VAX_DATE'], covid['ONSET_DATE']-covid['NUMDAYS'])

covid['VAX_DATE'] = np.where((pd.notnull(covid['VAX_DATE'])== True), covid['VAX_DATE'], covid['ONSET_DATE'])

current = covid['VAX_DATE'].isnull().sum()
print('ONSET_DATE컬럼을 통해', previous-current,'개의 결측값을 채웠으며, 현재 백신 접종 날짜의 결측치 갯수는',current,'개입니다.')

ValueError: ignored

VAERS에 각 환자의 투약 단계별 증상을 소팅해놓은 symptoms 데이터를 불러와서 died, disable, recovd 특성의 결측치를 채워보겠습니다.

1. 세 컬럼이 모두 비어있는 환자번호를 확인
2. symptoms 데이터에서 해당 환자의 증상 확인
3. 결측치 채우기

In [7]:
covid.RECOVD.unique()

array(['Y', 'N', 'U', nan, 'Obstructive sleep apnea', '2', ' chills"',
       'chills'], dtype=object)

In [ ]:
covid['result']  = covid['RECOVD']
covid['result'] = np.where(pd.notnull(covid['result']) == True |, df_2['C2'], df_2['C1'])

In [ ]:
symp_21 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4Hcd_jZN3fabc39Tn1dR62mbyscDDRGlvByJNBsijNSEJQ6JMBhBwcfACUNgYEGih_FcUTZcFTtrW/pub?output=csv') 
symp_20 = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRrjdwhIxgzkNi04EzJLeaimoKJm-CAug3Tf6uULhVXTUrxVo9mmokBeOtvGgyuOytiFo1oJzjCrtiR/pub?output=csv')

symp = pd.concat([symp_20,symp_21])

symp.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,855017,Arthralgia,22.1,Chills,22.1,Injection site pain,22.1,Pyrexia,22.1,NaN,NaN
1,855018,Chills,22.1,Fatigue,22.1,Hypertension,22.1,Hypoaesthesia,22.1,Injected limb mobility decreased,22.1
2,855018,Muscular weakness,22.1,Pain in extremity,22.1,Pyrexia,22.1,Tremor,22.1,Vertigo,22.1
3,855019,Pain,22.1,Pruritus,22.1,Rash,22.1,NaN,NaN,NaN,NaN
4,855020,Chills,22.1,Influenza like illness,22.1,Myalgia,22.1,Pain in extremity,22.1,Pyrexia,22.1


In [ ]:
symp = pd.merge(covid, symp, on = ['VAERS_ID'], how = 'inner')
id = symp.VAERS_ID

symps = []
for symp1, symp2, symp3, symp4, symp5 in zip(symp.SYMPTOM1, symp.SYMPTOM2, symp.SYMPTOM3, symp.SYMPTOM4, symp.SYMPTOM5):
  if symp5 != None:
    symps.append(symp5)
  else:
    if symp4 != None:
      symps.append(symp4)
    else:
      if symp3 != None:
        symps.append(symp3)
      else:
        if symp2 != None:
          symps.append(symp2)
        else:
          if symp1 != None:
            symps.append(symp1)
          else:
            symps.append('No adverse')

symptoms = pd.Series(symps)
symptoms = symptoms.value_counts().rename_axis('values').reset_index(name='counts')
symptoms = symptoms.sort_values('counts',ascending=False)
for symptom in symptoms.values:
  print(symptom)

['Pyrexia' 378]
['Pain' 339]
['Nausea' 277]
['Headache' 231]
['Pain in extremity' 202]
['Injection site pain' 151]
['Myalgia' 141]
['Fatigue' 130]
['Vomiting' 129]
['SARS-CoV-2 test negative' 109]
['Paraesthesia' 106]
['Dyspnoea' 106]
['Malaise' 99]
['Rash' 93]
['Palpitations' 80]
['Tachycardia' 78]
['Hyperhidrosis' 77]
['SARS-CoV-2 test positive' 77]
['Urticaria' 75]
['Pruritus' 72]
['Dizziness' 66]
['Hypoaesthesia' 65]
['Throat tightness' 58]
['Feeling abnormal' 58]
['Tremor' 57]
['Injection site swelling' 56]
['Heart rate increased' 56]
['Paraesthesia oral' 53]
['Throat irritation' 51]
['Oropharyngeal pain' 48]
['Laboratory test' 47]
['Unresponsive to stimuli' 45]
['Syncope' 43]
['Flushing' 43]
['Lymphadenopathy' 40]
['Metabolic function test' 37]
['Full blood count' 37]
['Swelling face' 37]
['Neck pain' 37]
['Peripheral swelling' 35]
['Impaired work ability' 35]
['SARS-CoV-2 test' 34]
['Sleep disorder' 34]
['Rhinorrhoea' 33]
['Swollen tongue' 31]
['Electrocardiogram normal' 31]
['M

In [ ]:
dead = []
disabled = []
for id, symp1, symp2, symp3, symp4, symp5 in zip(symp.VAERS_ID, symp.SYMPTOM1, symp.SYMPTOM2, symp.SYMPTOM3, symp.SYMPTOM4, symp.SYMPTOM5):
  symptoms = [symp1, symp2, symp3, symp4, symp5]
  if 'Death' in symptoms or 'Sudden cardiac death' in symptoms or 'Sudden death' in symptoms or 'Foetal death' in symptoms 
